In [1]:
import pandas as pd
import datetime
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
excel_file="/home/jian/Projects/Saatva/MMM/URL_models/input/URL_SM_input_tabs_data_JL_2019-05-06.xlsx"
excel_file=pd.ExcelFile(excel_file)
excel_file.sheet_names

tab_list=[x for x in excel_file.sheet_names if "Stat" not in x]
tab_list

['Home Page', 'Blogs', 'Other', '404 Pages']

In [3]:
df_data=excel_file.parse('Blogs')

DVs=df_data[['date','trans_net','Users','Sessions','ga:goal11Completions']]
DVs['Trans_Per_Usr']=DVs['trans_net']/DVs['Users']

df_ivs=df_data.copy()
df_ivs.drop(['date','trans_net','sales_net', 'Users', 'Sessions','ga:goal11Completions','Time on Page','Transactions'],axis=1,inplace=True)



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
dict_count_0={}
for col in df_ivs.columns.tolist():
    df_ivs[col]=df_ivs[col].astype(int)
    count_0=len([x for x in df_ivs[col].tolist() if x==0])
    dict_count_0.update({col:count_0})

In [5]:
stat_disc=pd.DataFrame(df_ivs.describe()).T.reset_index().rename(columns={"index":"Blog_URL"})
stat_disc['count_0']=stat_disc['Blog_URL'].apply(lambda x: dict_count_0[x])

In [6]:
stat_disc_exclude=stat_disc[stat_disc['count_0']>=365*0.95]
exclude_urls_list=stat_disc_exclude['Blog_URL'].tolist()

In [7]:
cleaned_df=df_ivs[[x for x in df_ivs.columns.tolist() if x not in exclude_urls_list]]

In [8]:
cleaned_df['date']=df_data['date']
cleaned_df=pd.merge(DVs,cleaned_df,on="date")
cleaned_df.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(365, 318)

In [9]:
cleaned_df.to_csv("/home/jian/Projects/Saatva/MMM/URL_models/input/Original_Blogs_Excluded_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


# PCA

In [10]:
df_ivs=cleaned_df[[x for x in cleaned_df.columns.tolist() if x not in ['date','trans_net','Users','Sessions','ga:goal11Completions','Trans_Per_Usr']]]
X = cleaned_df[[x for x in cleaned_df.columns.tolist() if x not in ['date','trans_net','Users','Sessions','ga:goal11Completions','Trans_Per_Usr']]].values

X_std = StandardScaler().fit_transform(X)
X_std.shape

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


(365, 312)

In [11]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)

cov_mat = np.cov(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

Everything ok!


In [12]:

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')

i_counter=0
for i in eig_pairs:
    if i[0]>1:
        # print(i[0])
        i_counter+=1
print("i_counter",i_counter)

Eigenvalues in descending order:
i_counter 97


In [13]:
to_hstack=[eig_pairs[i][1].reshape(312,1) for i in range(97)]
matrix_w = np.hstack(to_hstack)

print('Matrix W:\n', matrix_w)

Matrix W:
 [[-0.03011189  0.02887082  0.0567814  ..., -0.0532146  -0.02095707
   0.00410964]
 [-0.03969654  0.05573523  0.01795734 ..., -0.08395556 -0.01006451
   0.03250126]
 [ 0.0166415  -0.0220699   0.00872927 ...,  0.01281653  0.00905728
  -0.0419585 ]
 ..., 
 [-0.15746295  0.00601838 -0.01490649 ..., -0.0121412  -0.01806139
  -0.00218022]
 [ 0.00577714 -0.01789422  0.03884227 ..., -0.11944565 -0.01993597
   0.00142605]
 [ 0.00510526 -0.01194857  0.04774608 ...,  0.04748559 -0.07759031
  -0.04546095]]


In [14]:
Y = X_std.dot(matrix_w)

In [15]:
DVs=cleaned_df[['date','trans_net','Users','Sessions','ga:goal11Completions']]
DVs['Trans_Per_Usr']=DVs['trans_net']/DVs['Users']

# 0
df_cov_mat=pd.DataFrame(cov_mat)
df_cov_mat.index=df_ivs.columns.tolist()
df_cov_mat.columns=df_ivs.columns.tolist()

# 1
df_eigen_values=pd.DataFrame(columns={"component_count","eigenvalue"})
for i in range(len(eig_pairs)):
    df=pd.DataFrame({"component_count":i+1,"eigenvalue":eig_pairs[i][0]},index=[i])
    df_eigen_values=df_eigen_values.append(df)
# 2
df_X_std=pd.DataFrame(X_std)
df_X_std.columns=df_ivs.columns.tolist()
df_X_std.index=df_data['date']
df_X_std=df_X_std.reset_index()
# 3
matrics_weight=pd.DataFrame(matrix_w)
# 4
input_transformed=pd.DataFrame(Y)
input_transformed.index=df_data['date']
input_transformed.columns=["Component_"+str(x+1) for x in input_transformed.columns.tolist()]
input_transformed=input_transformed.reset_index()

input_transformed=pd.merge(input_transformed,DVs,on="date")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
input_transformed.to_csv("/home/jian/Projects/Saatva/MMM/URL_models/input/PCA_Blog_Ecluded_MMM_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

writer=pd.ExcelWriter("/home/jian/Projects/Saatva/MMM/URL_models/input/PCA_Blog_MMM_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_data.to_excel(writer,"Raw_Blogs_data",index=False)
cleaned_df.to_excel(writer,"Excluded_95_0_Blogs_data",index=False)
df_cov_mat.to_excel(writer,"cov_mat",index=False)
df_eigen_values.to_excel(writer,"eigen_values",index=False)
df_X_std.to_excel(writer,"df_IVs_std",index=False)
matrics_weight.to_excel(writer,"matrics_weight",index=False)
input_transformed.to_excel(writer,"input_transformed",index=False)
writer.save()



In [18]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=97)
Y_sklearn = sklearn_pca.fit_transform(X_std)

In [21]:
input_transformed.tail(4)

,date,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,...,Component_93,Component_94,Component_95,Component_96,Component_97,trans_net,Users,Sessions,ga:goal11Completions,Trans_Per_Usr
361,2018-12-28,-12.150776,4.553925,-4.507620,-2.471403,4.233136,-0.569572,2.548848,0.140244,-0.751662,...,-1.755470,-0.184880,-0.143888,0.333531,-2.018870,204,27175,31895,500,0.007507
362,2018-12-29,-12.619738,6.335100,-3.714456,-1.654520,5.939165,1.119476,2.633794,1.273394,-1.314645,...,0.084378,3.133645,-1.063711,1.436055,0.472943,239,28337,33346,544,0.008434
363,2018-12-30,-14.840849,5.064564,-6.349244,-2.551707,7.375521,-0.688136,4.776907,-1.022006,0.600109,...,0.350509,-0.586233,1.449566,1.325331,1.038980,221,28716,34009,555,0.007696
364,2018-12-31,-12.469221,5.034077,-5.867042,-3.580926,3.606432,-0.708219,6.644007,0.049611,-1.571092,...,-0.206035,0.606820,0.381060,-0.576743,0.314634,258,24087,28142,452,0.010711


In [22]:
pd.DataFrame(Y_sklearn).tail(4)

# Looks_Alright

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
361,12.150776,-4.553925,4.507620,2.471403,-4.233136,-0.569572,-2.548848,0.140244,0.751662,-1.780143,...,0.287408,1.426941,-1.242240,-2.030988,1.230140,1.755470,-0.184880,-0.143888,-0.333531,-2.018870
362,12.619738,-6.335100,3.714456,1.654520,-5.939165,1.119476,-2.633794,1.273394,1.314645,-1.065144,...,-0.123561,2.139012,0.450934,0.028680,1.869606,-0.084378,3.133645,-1.063711,-1.436055,0.472943
363,14.840849,-5.064564,6.349244,2.551707,-7.375521,-0.688136,-4.776907,-1.022006,-0.600109,-2.982750,...,-2.129806,-0.225863,0.404393,1.859784,2.196047,-0.350509,-0.586233,1.449566,-1.325331,1.038980
364,12.469221,-5.034077,5.867042,3.580926,-3.606432,-0.708219,-6.644007,0.049611,1.571092,-2.809859,...,3.170263,0.358848,-0.923918,0.050193,1.449146,0.206035,0.606820,0.381060,0.576743,0.314634


In [43]:
chech_array=np.round(abs(pd.DataFrame(Y_sklearn).values),6)==np.round(abs(input_transformed[[x for x in input_transformed.columns.tolist() if "Component" in x]].values),6)

all_unique=[]
for i in range(len(chech_array)):
    QC=set((np.round(abs(pd.DataFrame(Y_sklearn).values),6)==np.round(abs(input_transformed[[x for x in input_transformed.columns.tolist() if "Component" in x]].values),6))[1])
    # print(QC)
    all_unique=all_unique+[str(QC)]
set(all_unique)

{'{True}'}

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

# EFA

In [48]:
import pandas as pd
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
import datetime

import pandas as pd
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
import datetime

In [46]:
df_ivs = cleaned_df[[x for x in cleaned_df.columns.tolist() if x not in ['date','trans_net','Users','Sessions','ga:goal11Completions','Trans_Per_Usr']]]
df_ivs.shape

(365, 312)

In [47]:
chi_square_value,p_value=calculate_bartlett_sphericity(df_ivs)
chi_square_value, p_value

NameError: name 'calculate_bartlett_sphericity' is not defined